<img src="data/images/lecture-notebook-header.png" />

# k-NN Classification

The k-Nearest Neighbors (k-NN) algorithm is a simple and intuitive machine learning algorithm used for both classification and regression tasks. It operates based on the idea that similar data points are likely to belong to the same class or have similar characteristics. Here's an overview of the k-Nearest Neighbors algorithm and its application in text document classification:

* **Algorithm Overview:** k-NN makes predictions by comparing a new, unseen data point (in this case, a text document) to the labeled data points (documents with known categories) in its training dataset. It calculates the similarity (often using distance metrics like Euclidean distance or cosine similarity) between the new document and all the documents in the training set.

* **k Neighbors:** The "k" in k-NN represents the number of nearest neighbors (documents) to consider. To classify a new document, the algorithm identifies the k nearest neighbors in the training data based on their similarity to the new document.

* **Voting or Weighted Averaging:** For classification tasks, once the k nearest neighbors are identified, k-NN uses either a voting mechanism or weighted averaging of their labels to determine the class of the new document. In voting, the class that occurs most frequently among the k neighbors is assigned to the new document. In weighted averaging, the neighbors' labels are weighted by their proximity or similarity to the new document.

* **Text Document Classification:** For text document classification, the k-NN algorithm can be applied by representing documents as numerical vectors using techniques like TF-IDF (Term Frequency-Inverse Document Frequency) or word embeddings. Each document becomes a point in a high-dimensional space, and the algorithm calculates distances or similarities between these points to find the nearest neighbors.

* **Parameter Tuning:** The choice of k (the number of neighbors) is crucial and affects the model's performance. A smaller k may lead to more complex decision boundaries and could be more sensitive to outliers, while a larger k might oversmooth decision boundaries.

* **Scalability and Efficiency:** While k-NN is straightforward and easy to understand, it might not be very efficient, especially with large datasets, as it requires calculating distances to every point in the training set for each prediction.

In summary, k-Nearest Neighbors is a versatile algorithm suitable for text document classification by measuring similarities between documents. It can be effective for smaller datasets or when the decision boundaries between classes are relatively simple. However, its performance might degrade with high-dimensional data or when the dataset size becomes substantial due to computational complexity.

## Setting up the Notebook

### Import Required packages

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.pipeline import Pipeline

from tqdm import tqdm

## Preparing the Data

For this notebook, we use a simple dataset for sentiment classification. This dataset consists of 10,662 sentences, where 50% of the sentences are labeled 1 (positive), and 50% of the sentences are labeled -1 (negative).

### Loading Sentence/Label Pairs from File


In [ ]:
sentences, labels = [], []

with open("data/datasets/sentence-polarities/sentence-polarities.csv") as file:
    for line in file:
        line = line.strip()
        sentence, label = line.split("\t")
        sentences.append(sentence)
        labels.append(int(label))  
        
print("Total number of sentences: {}".format(len(sentences)))

### Create Training & Test Set

To evaluate any classifier, we need to split our dataset into a training and a test set. With the method `train_test_split()` this is very easy to do; this method also shuffles the dataset by default, which is important for this example, since the dataset file is ordered with all positive sentences coming first. In the example below, we set the size of the test set to 20%.


In [ ]:
# Split sentences and labels into training and test set with a test set size of 20%
sentences_train, sentences_test, labels_train, labels_test = train_test_split(sentences, labels, test_size=0.2, random_state=42)

# We can directly convert the numerical class labels from lists to numpy arrays
y_train = np.asarray(labels_train)
y_test = np.asarray(labels_test)

print("Size of training set: {}".format(len(sentences_train)))
print("Size of test set: {}".format(len(sentences_test)))

## Training & Testing a k-NN Classifier

Let's first have a look at how to train a k-NN classifier with the minimum number of steps. This includes that we simply pick some values for all hyperparameters which in the case of k-NN is simply the number of nearest neighbors $k$. Of course, we should also not forget that there are different hyperparameters when it comes to converting our corpus into the Document-Term Matrix. But let's just assume the following setting for this.


In [ ]:
# Create Document-Term Matrix for differen n-gram sizes
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), max_features=10000)

X_train = tfidf_vectorizer.fit_transform(sentences_train)
X_test = tfidf_vectorizer.transform(sentences_test)

Using the training data, we can train a k-NN classifier with a single line of code


In [ ]:
knn = KNeighborsClassifier(n_neighbors=20).fit(X_train, y_train)

Once trained, we can predict the class labels for the document vectors in our test set.

In [ ]:
y_pred = knn.predict(X_test)

`y_pred` now contains the 2,133 predicted labels that we can compare with the ground truth labels from the test set. scikit-learn provides methods to easily calculate all the important metrics we covered in the lecture. Since we only have to class labels (i.e., binary classification), we do not have to set the `average` parameter to indicate micro or macro averaging.

In [ ]:
precision = metrics.precision_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)
f1 = metrics.f1_score(y_test, y_pred)

print("Precison: {:.3f}".format(precision))
print("Recall:   {:.3f}".format(recall))
print("F1 score: {:.3f}".format(f1))

scikit-learn also provides a method `classification_report()` for a more detailed description of the results, showing a breakdown of the precision, recall, and f1 scores broken down for each class.

In [ ]:
print(metrics.classification_report(y_test, y_pred))

## Hyperparameter Tuning

In the example above, we simply set $k=20$ in the hopes that this will yield good results. In practice, of course, we are interested in finding the best values for $k$ systematically and not just guessing it. This is where hyperparameter tuning and cross validation comes into play.

### Selecting the Right K (only)

Let's first consider $k$ as the only hyperparameter. To find its best value, we have to try different choices, perform cross validation for the choice of $k$, and keep track of the results. The loop in the code cell below accomplishes this. Note the method `cross_val_score()` from scikit-learn that automatically performs k-fold cross validation with a single line of code. In the example below, we perform 10-fold cross validations (`cv=10`). Since we get 10 f1 scores, one for each fold, we compute the final f1 score as the average over the scores of all folds.


In [ ]:
max_num_neighbors = 100

f1_scores = []

# Loop over all odd values between 1 and max_num_neighbors+1
for k in tqdm(range(1, max_num_neighbors+1, 2)):
    knn = KNeighborsClassifier(n_neighbors=k)
    
    scores = cross_val_score(knn, X_train, y_train, cv=10, scoring="f1")
    mean_score = np.mean(scores)
    
    f1_scores.append((k, mean_score))

In the code cell above, we simply store all combinations of $k$ values and their corresponding f1 scores. This allows us to easily plot the result.


In [ ]:
plt.figure()
plt.plot([s[0] for s in f1_scores], [s[1] for s in f1_scores], lw=3)
font_axes = {'family':'serif','color':'black','size':16}
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("Number of Neighbors k", fontdict=font_axes)
plt.ylabel("F1 Score", fontdict=font_axes)
plt.tight_layout()
plt.show()

From the plot we can see that the f1 scores initially improve quite quickly, but around $k=40$ seem starting to converge, and later even slightly drop again.

We could now pick our choice of $k$ by eye-balling the plot above, or we can find the best choice of $k$ also programmatically. With the data we already have, we can first find the index position in the result list that corresponds to the highest f1 scores. We can then look up the respective $k$ values. The code cell below accomplishes this:


In [ ]:
k_values = np.asarray([ t[0] for t in f1_scores ])
f1_values = np.asarray([ t[1] for t in f1_scores ])

# Find the indices referring to the largest f1 scores
indices_sorted = np.argsort(f1_values)[::-1]

# Show k values sorted w.r.t. the f1 scores in descending order
print(k_values[indices_sorted])

This results tells us that for $k=67$ we will get the largest f1 score. However, we can also see from the plot that all top results are rather similar, and our dataset is not very large to make a really good judgment. Nevertheless, let's work with $k=67$. Having identified this as our best choice for $k$, we can now train a k-NN classifier with this value over the whole training data, and finally calculate the f1 score using the test data.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=67).fit(X_train, y_train)

y_pred = knn.predict(X_test)

precision = metrics.precision_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)
f1 = metrics.f1_score(y_test, y_pred)

print("Precison: {:.3f}".format(precision))
print("Recall:   {:.3f}".format(recall))
print("F1 score: {:.3f}".format(f1))

This looks obvious better then our initial choice of $k=20$ above.

### Selecting the Right K and Maximum N-Gram Size

So far, we used the same document vectors for finding the best value of $k$. However, we know that we already have options when it comes to feature extractions (you can check out all the input parameters of [`TfidfVectorizer`](https://www.google.com/search?channel=fs&client=ubuntu&q=sklearn+tfidfvectorizer)). In principle, we could consider all possible parameters. However, the search space would quickly explode. So let's just consider the maximum size of n-grams. To further limit the number of runs, we further only consider a subset of possible values for $k$.


In [ ]:
k_sizes = [7, 17, 27, 37, 47, 57, 67, 77, 87, 97 ]
#k_sizes = [7, 17, 27]

max_ngram_size = 5
#max_ngram_size = 3
num_k = len(k_sizes)

# Number runs = number traing/test a k-NN classifier
num_runs = max_ngram_size * num_k

# numpy array to keep track of all results
knn_results = np.zeros((max_ngram_size, num_k))

with tqdm(total=num_runs) as pbar:
    for i, ngram in enumerate(range(1, max_ngram_size+1)):
        # Create Document-Term Matrix for different n-gram sizes
        tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, ngram), max_features=20000)
        X_train = tfidf_vectorizer.fit_transform(sentences_train)
        X_test = tfidf_vectorizer.transform(sentences_test)
        # Train & test model using cross validation
        for j, k in enumerate(k_sizes):
            knn = KNeighborsClassifier(n_neighbors=k)
            scores = cross_val_score(knn, X_train, y_train, cv=10, scoring="f1")
            mean_score = np.mean(scores)
            knn_results[i,j] = mean_score
            pbar.update(1)

Now that we performed hyperparameter tuning for 2 parameters at the same time, a simple line plot is no longer suitable. The code cell below plots a heatmap, indicating the combination of maximum n-gram size and $k$ gave us the highest f1 score(s). The Python package `seaborn` makes plotting heatmaps quite easy.

In [ ]:
# Use the heatmap function from the seaborn package
plt.figure()

sns.heatmap(knn_results, annot=True, cmap="crest", xticklabels=k_sizes, yticklabels=list(range(1,max_ngram_size+1)))

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Maximum N-Gram Size', fontsize=16)
plt.xlabel('Number of Neighbors (k)', fontsize=16)

plt.show()

When looking at this plot, it seems that using just unigrams and our initially best value of $k=67$ seems to do just fine. Of course, in principle, we would need to check other parameters such as `max_features` or `min_df` as well. Depending on the runtime of a single training and cross validation step, we might not try all possible combinations. Instead, we would tune 1-2 parameters at the same time, keeping all other parameters fixed. Once having identified meaningful values for those parameters, we keep this fixed and tune another choice of 1-2 parameters.

## Pipelines & Grid Search

Hyperparameter tuning is a quite important step, but the previous example has shown that it can be quite tedious. However, note that we basically tried all possible combinations for certain sets of parameter values. And since we were tuning 2 parameters, we required 2 nested loops. Thus, if we would tune $N$ parameters at the same time, we would need to have $N$ nested loops. Luckily, scikit-learn makes this much easier using [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

Since the parameters we would like to tune refer to 2 different components -- the vectorizer and the classifier -- we also need a [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) to combine both components into a single model. Let's do this first:


In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knn', KNeighborsClassifier()),
])

Now we can define the search space, by providing the set of values for the hyperparameters we want to consider. See how the identifier of the parameters are a combination of the name in the pipeline (here: `tfidf` and `knn`) and the name of the parameter in the respective class. For example, `tfidf__max_df` refers to the `max_df` parameter of the `TfidfVectorizer`.

In [ ]:
parameters = {
    'tfidf__max_df': (0.75, 1.0),
    'tfidf__max_features': (5000, 10000),
    'tfidf__ngram_range': ((1, 1), (1, 2)),
    'knn__n_neighbors': [57, 67, 77]
}

Now we can use `GridSearchCV` to check all possible combinations of parameter values. Of course, we kept the number of possible values rather small to avoid overly run times here. Note that for any parameter not listed above (e.g., `min_df` of the vectorizer) the default value is used.

In [ ]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=1, verbose=2, cv=5)

grid_search = grid_search.fit(sentences_train, labels_train)

Once the `GridSearchCV` has checked all possible parameter combinations, we can read out the best combination as follows:

In [ ]:
print(grid_search.best_params_)

---

## Summary

The k-Nearest Neighbors (k-NN) algorithm is a straightforward yet powerful method used in text classification tasks. It operates on the principle of similarity, making predictions for new text documents based on their resemblance to known labeled documents in a training set.

In text classification, k-NN treats each document as a data point in a high-dimensional space, where the dimensions represent features such as word frequencies or TF-IDF values. When a new document needs classification, k-NN calculates its similarity to all documents in the training set using distance metrics like Euclidean distance or cosine similarity. The algorithm identifies the k most similar documents (neighbors) to the new document. These neighbors contribute to the classification decision by either voting (assigning the class label most common among the neighbors) or weighted averaging of their labels based on their proximity to the new document.

One of the strengths of k-NN in text classification lies in its simplicity and ease of implementation. It doesn't require assumptions about the underlying data distribution and can handle multi-class classification effectively. However, its performance can be affected by the choice of k—the number of neighbors considered—where a smaller k might lead to more complex decision boundaries, while a larger k might oversimplify them. Additionally, k-NN can face challenges with scalability and computational efficiency, especially with large datasets, as it needs to compute distances to all training instances for each prediction.

Overall, k-Nearest Neighbors is a valuable algorithm for text classification, providing a intuitive way to categorize new documents based on their similarity to existing labeled documents in the training set. Its applicability in smaller datasets and its simplicity make it a useful baseline method in text classification tasks.